In [1]:
# Importing libraries
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
# Importing model libraries
import xgboost as xgb

In [3]:
# Read .csv file with company names

df = pd.read_csv('GREATEC.csv')

# Changing variable into a dataframe
df= df[['c','d','h','l','o','v']]
df['d'] = pd.to_datetime(df['d'], dayfirst= True )
df.set_index('d',inplace=True,drop=True)
df.index.name = None

In [4]:
# Get price changes between 1Day gap

In [5]:
day1_different =df.diff().dropna()

In [6]:
for col in day1_different.columns:
    for i in range(1,2):
        day1_different[col+str(i)+'_day_ago'] = day1_different[col].shift(i)

day1_different.dropna(inplace=True)

In [7]:
# Drop variable that have high correlation 
day1_different.drop(['c','o','v','h','h1_day_ago', 'l', 'l1_day_ago'],axis='columns', inplace=True)

In [8]:
#Train/Test Split
#Cut off the data after 13-Jan-2021 to use as our validation set.
split_date = '13-Jan-2021'
stock_train = day1_different.loc[day1_different.index <= split_date].copy()
stock_test = day1_different.loc[day1_different.index > split_date].copy()

In [9]:
#Create Time Series Features
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    
    df['c1_day_ago'] = df.c1_day_ago
    df['o1_day_ago'] = df.o1_day_ago
    df['v1_day_ago'] = df.v1_day_ago
  
    
    
    X = df[['c1_day_ago','o1_day_ago','v1_day_ago']]
    if label:
        y = df[label]
        return X, y
    return X

In [10]:
X_train, y_train = create_features(stock_train,label='c1_day_ago')
X_test, y_test = create_features(stock_test ,label='c1_day_ago')

In [11]:
# Model specification
reg = xgb.XGBRegressor(n_estimators=1000)

# Training the model
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=40)

[0]	validation_0-rmse:0.34771	validation_1-rmse:0.26882
[1]	validation_0-rmse:0.24494	validation_1-rmse:0.18367
[2]	validation_0-rmse:0.17264	validation_1-rmse:0.12403
[3]	validation_0-rmse:0.12186	validation_1-rmse:0.08643
[4]	validation_0-rmse:0.08618	validation_1-rmse:0.06244
[5]	validation_0-rmse:0.06102	validation_1-rmse:0.04626
[6]	validation_0-rmse:0.04335	validation_1-rmse:0.03764
[7]	validation_0-rmse:0.03097	validation_1-rmse:0.03301
[8]	validation_0-rmse:0.02226	validation_1-rmse:0.03245
[9]	validation_0-rmse:0.01617	validation_1-rmse:0.03275
[10]	validation_0-rmse:0.01190	validation_1-rmse:0.03267
[11]	validation_0-rmse:0.00889	validation_1-rmse:0.03335
[12]	validation_0-rmse:0.00679	validation_1-rmse:0.03348
[13]	validation_0-rmse:0.00531	validation_1-rmse:0.03374
[14]	validation_0-rmse:0.00426	validation_1-rmse:0.03387
[15]	validation_0-rmse:0.00352	validation_1-rmse:0.03419
[16]	validation_0-rmse:0.00296	validation_1-rmse:0.03477
[17]	validation_0-rmse:0.00258	validation

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
#Forecast on Test Set
reg.predict(X_test)

array([ 0.20312083,  0.4124382 ,  0.06151441,  0.14245974, -0.07733961],
      dtype=float32)